# 02 — RNN pooling fix (SmartWatch Gestures)

# Doel
De baseline gebruikt een naïeve sequentierepresentatie door alleen de **laatste timestep** van de (gepaddede) sequentie te gebruiken voor classificatie.  
Omdat sequenties **variabele lengte** hebben en op batch-niveau worden **gepaddede**, kan deze keuze informatieverlies veroorzaken of padding-effecten introduceren.

# Hypothese
Als we de sequentierepresentatie vervangen door **pooling over tijd** (bijv. gemiddelde over alle timesteps), dan wordt het model robuuster voor variabele sequentielengtes en verwachten we een **hogere en/of stabielere validatie-accuracy**.

# Experiment
We houden dataset, preprocessor, TrainerSettings en optimizer gelijk, en vergelijken:
- Baseline representatie: laatste timestep
- Pooling representatie: mean pooling over tijd

Resultaten worden gelogd in MLflow zodat runs direct vergelijkbaar zijn.


In [ ]:
from pathlib import Path

import torch

from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy